In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time

In [126]:
df = pd.read_csv('in-vehicle-coupon-recommendation.csv')
df.head()


,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [127]:
df.shape

(12684, 26)

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

In [129]:
le = LabelEncoder()

In [130]:
categorical_features = ['destination', 'passanger', 'weather', 'time', 'coupon', 'expiration',
                        'gender', 'age', 'maritalStatus', 'education', 'occupation', 'income',
                        'car', 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20',
                        'Restaurant20To50']

for feature in categorical_features:
    df[f'{feature}_clean'] = le.fit_transform(df[feature])
    df.drop(feature, inplace=True, axis=1)

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   temperature                 12684 non-null  int64
 1   has_children                12684 non-null  int64
 2   toCoupon_GEQ5min            12684 non-null  int64
 3   toCoupon_GEQ15min           12684 non-null  int64
 4   toCoupon_GEQ25min           12684 non-null  int64
 5   direction_same              12684 non-null  int64
 6   direction_opp               12684 non-null  int64
 7   Y                           12684 non-null  int64
 8   destination_clean           12684 non-null  int64
 9   passanger_clean             12684 non-null  int64
 10  weather_clean               12684 non-null  int64
 11  time_clean                  12684 non-null  int64
 12  coupon_clean                12684 non-null  int64
 13  expiration_clean            12684 non-null  int64
 14  gender

In [132]:
X = df.drop('Y', axis=1)
y = df['Y']

In [133]:
X

,temperature,has_children,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,destination_clean,passanger_clean,weather_clean,...,maritalStatus_clean,education_clean,occupation_clean,income_clean,car_clean,Bar_clean,CoffeeHouse_clean,CarryAway_clean,RestaurantLessThan20_clean,Restaurant20To50_clean
0,55,1,1,0,0,0,1,1,0,2,...,3,5,24,3,5,4,4,5,1,0
1,80,1,1,0,0,0,1,1,1,2,...,3,5,24,3,5,4,4,5,1,0
2,80,1,1,1,0,0,1,1,1,2,...,3,5,24,3,5,4,4,5,1,0
3,80,1,1,1,0,0,1,1,1,2,...,3,5,24,3,5,4,4,5,1,0
4,80,1,1,1,0,0,1,1,1,2,...,3,5,24,3,5,4,4,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,55,0,1,0,0,1,0,0,3,0,...,2,1,21,6,5,4,4,0,1,0
12680,55,0,1,0,0,0,1,2,0,0,...,2,1,21,6,5,4,4,0,1,0
12681,30,0,1,0,0,1,0,2,0,1,...,2,1,21,6,5,4,4,0,1,0
12682,30,0,1,1,1,0,1,2,0,1,...,2,1,21,6,5,4,4,0,1,0


In [134]:
y

0        1
1        0
2        1
3        0
4        0
        ..
12679    1
12680    1
12681    0
12682    0
12683    0
Name: Y, Length: 12684, dtype: int64

In [135]:
imputer = SimpleImputer(strategy='mean')

In [136]:
imputer = imputer.fit(X)

In [137]:
X = imputer.fit_transform(X)

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [139]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1, 1)

In [140]:
lr = LogisticRegression(random_state=42)

lr_time_start = time.perf_counter()

lr.fit(X_train, y_train)
lr.predict(X_test)

lr_time_end = time.perf_counter()

lr_time = lr_time_end - lr_time_start

/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [141]:
knn = KNeighborsClassifier(n_neighbors=2)

knn_time_start = time.perf_counter()

knn.fit(X_train, y_train)
knn.predict(X_test)

knn_time_end = time.perf_counter()

knn_time = knn_time_end - knn_time_start

/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [142]:
svc = SVC(random_state=42)

svc_time_start = time.perf_counter()

svc.fit(X_train, y_train)
svc.predict(X_test)

svc_time_end = time.perf_counter()

svc_time = svc_time_end - svc_time_start

/Users/daniyarkurmanbayev/Documents/GBC/mlenv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [143]:
dtf = DecisionTreeClassifier(random_state=42)

dtf_time_start = time.perf_counter()

dtf.fit(X_train, y_train)
dtf.predict(X_test)

dtf_time_end = time.perf_counter()

dtf_time = dtf_time_end - dtf_time_start

In [144]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

rfc_time_start = time.perf_counter()

rfc.fit(X_train, y_train)
rfc.predict(X_test)

rfc_time_end = time.perf_counter()

rfc_time = rfc_time_end - rfc_time_start

/var/folders/1b/216kmv2s3731399_bt742lgw0000gn/T/ipykernel_13358/3378031463.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


In [145]:
times_names = ['lr_time', 'knn_time', 'svc_time', 'dtf_time', 'rfc_time']
times = [lr_time, knn_time, svc_time, dtf_time, rfc_time]

times_sorted = np.argsort(times)
np.array(times_names)[times_sorted[::-1]]



array(['svc_time', 'rfc_time', 'knn_time', 'lr_time', 'dtf_time'],
      dtype='<U8')